In [1]:
!wget https://raw.githubusercontent.com/BKHV/risk_models/master/data/PD-data-train.csv
!wget https://raw.githubusercontent.com/BKHV/risk_models/master/data/PD-data-test.csv
!wget https://raw.githubusercontent.com/BKHV/risk_models/master/data/PD-data-desc.csv

--2023-02-01 14:34:50--  https://raw.githubusercontent.com/BKHV/risk_models/master/data/PD-data-train.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5329268 (5.1M) [text/plain]
Saving to: ‘PD-data-train.csv.1’

PD-data-train.csv.1 100%[===================>]   5.08M  --.-KB/s    in 0.09s   

2023-02-01 14:34:50 (56.2 MB/s) - ‘PD-data-train.csv.1’ saved [5329268/5329268]

--2023-02-01 14:34:51--  https://raw.githubusercontent.com/BKHV/risk_models/master/data/PD-data-test.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 28259

In [2]:
import pandas as pd
from sklearn.metrics import f1_score
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn import tree, preprocessing
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import cross_val_score, train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score
import numpy as np
from sklearn.feature_selection import f_classif, chi2, SelectKBest

In [3]:
train_df = pd.read_csv('PD-data-train.csv', sep=';')
test_df = pd.read_csv('PD-data-test.csv', sep=';')
desc_df = pd.read_csv('PD-data-desc.csv', sep=';')

In [4]:
test_df

,record_id,ar_revenue,ar_total_expenses,ar_sale_cost,ar_selling_expenses,ar_management_expenses,ar_sale_profit,ar_balance_of_rvns_and_expns,ar_profit_before_tax,ar_taxes,...,ogrn_age,adr_actual_age,head_actual_age,cap_actual_age,ul_staff_range,ul_capital_sum,ul_founders_cnt,ul_branch_cnt,ul_strategic_flg,ul_systematizing_flg
0,196,39103000.0,38913000.0,38913000.0,0.0,0.0,190000.0,141000.0,331000.0,284000.0,...,135,3,3,3,[1-100],100000.0,1,0,0,0
1,1196,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,138,5,5,5,[1-100],100000.0,1,0,0,0
2,2813,54174000.0,50929000.0,50929000.0,0.0,0.0,3245000.0,-1278000.0,1967000.0,500000.0,...,138,20,20,20,[1-100],250000.0,2,0,0,0
3,4385,1904000.0,1679000.0,1679000.0,0.0,0.0,225000.0,-88000.0,137000.0,0.0,...,139,9,9,9,[1-100],10000.0,2,0,0,0
4,6479,225584000.0,210685000.0,210685000.0,0.0,0.0,14899000.0,-12715000.0,2184000.0,445000.0,...,139,3,3,3,(100-500],1700598.0,84,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,265577,21287000.0,18838000.0,16227000.0,2611000.0,0.0,2449000.0,-123000.0,2326000.0,483000.0,...,47,32,32,32,[1-100],10000.0,1,0,0,0
196,266648,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,140,6,6,6,[1-100],0.0,1,0,0,0
197,266849,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,22,7,7,7,[1-100],20000.0,1,0,0,0
198,267566,759009000.0,726682000.0,551903000.0,18323000.0,156456000.0,32327000.0,-23281000.0,9046000.0,431000.0,...,140,31,31,31,[1-100],100000.0,2,0,0,0


In [5]:
train_df

,record_id,ar_revenue,ar_total_expenses,ar_sale_cost,ar_selling_expenses,ar_management_expenses,ar_sale_profit,ar_balance_of_rvns_and_expns,ar_profit_before_tax,ar_taxes,...,adr_actual_age,head_actual_age,cap_actual_age,ul_staff_range,ul_capital_sum,ul_founders_cnt,ul_branch_cnt,ul_strategic_flg,ul_systematizing_flg,default_12m
0,7,52996000.0,48344000.0,48344000.0,0.0,0.0,4652000.0,-2608000.0,2044000.0,846000.0,...,3,3,3,[1-100],50000.0,2,0,0,0,0
1,13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,12,12,12,[1-100],10000.0,5,0,0,0,0
2,18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,28,28,28,[1-100],10000.0,2,0,0,0,0
3,22,32049000.0,29568000.0,29568000.0,0.0,0.0,2481000.0,-260000.0,2221000.0,302000.0,...,8,8,8,[1-100],300000.0,1,0,0,0,0
4,23,16458000.0,14935000.0,14935000.0,0.0,0.0,1523000.0,-274000.0,1249000.0,538000.0,...,27,27,27,[1-100],10000.0,2,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32390,274590,28264000.0,27911000.0,27911000.0,0.0,0.0,353000.0,-68000.0,285000.0,57000.0,...,5,5,5,[1-100],10000.0,1,0,0,0,0
32391,274593,25017000.0,24381000.0,24381000.0,0.0,0.0,636000.0,0.0,636000.0,0.0,...,25,25,25,[1-100],600000.0,1,0,0,0,0
32392,274602,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,11,11,11,[1-100],10000.0,1,0,0,0,0
32393,274611,136162000.0,135088000.0,135088000.0,0.0,0.0,1074000.0,-719000.0,355000.0,267000.0,...,13,13,13,(100-500],11000.0,1,0,0,0,1


In [6]:
desc_df

,field,desc_eng,desc_rus
0,ar_revenue,Sales revenue (at the end of the last year),Выручка от реализации (на конец последнего года)
1,ar_total_expenses,Total expenses for the last year,Общие расходы за последний год
2,ar_sale_cost,Cost of sales (at the end of the last year),Себестоимость продаж (на конец последнего года)
3,ar_selling_expenses,Selling expenses (at the end of the last year),Коммерческие расходы (на конец последнего года)
4,ar_management_expenses,Administrative expenses (at the end of the las...,Управленческие расходы (на конец последнего года)
5,ar_sale_profit,Profit from sales (at the end of the last year),Прибыль от реализации (на конец последнего года)
6,ar_balance_of_rvns_and_expns,Balance of income and expenses (at the end of ...,Баланс доходов и расходов (на конец последнего...
7,ar_profit_before_tax,Gross Profit (end of last year),Валовая прибыль (на конец последнего года)
8,ar_taxes,Current income tax (at the end of the last year),Текущий на налог на прибыль (на конец последне...
9,ar_other_profit_and_losses,Other income and expenses (at the end of the l...,Прочие доходы и расходы (на конец последнего г...


In [7]:

train_df['losses=margin'] = np.where(train_df['ar_net_profit']==train_df['ab_losses'], 1, 0)
test_df['losses=margin'] = np.where(test_df['ar_net_profit']==test_df['ab_losses'], 1, 0)


In [8]:
from sklearn.preprocessing import LabelEncoder
train_df['ul_staff_range'] = LabelEncoder().fit_transform(train_df['ul_staff_range'])
test_df['ul_staff_range'] = LabelEncoder().fit_transform(test_df['ul_staff_range'])

#Разделим тренировочный датафрейм на два - с финансовыми показателями и без (в датафрейме без фин показателей в два раза чаще встречается дефолт)

In [9]:
features = ['bus_age',	
                     'ul_staff_range',
                     'ul_founders_cnt',
                     'ul_capital_sum',
                     'head_actual_age']
a = train_df[train_df.isnull().any(1)]
without_ar_df = a.loc[:, features]
without_ar_target = a['default_12m']
without_ar_df

,bus_age,ul_staff_range,ul_founders_cnt,ul_capital_sum,head_actual_age
1,256,2,5,10000.0,12
2,33,2,2,10000.0,28
5,100,2,1,350000.0,5
6,266,2,3,35000.0,7
7,31,2,2,10000.0,30
...,...,...,...,...,...
32383,136,2,2,9408.0,23
32384,2,2,1,10000.0,2
32387,237,2,18,161799.0,2
32389,24,2,4,10000.0,3


In [10]:
train_df.columns

Index(['record_id', 'ar_revenue', 'ar_total_expenses', 'ar_sale_cost',
       'ar_selling_expenses', 'ar_management_expenses', 'ar_sale_profit',
       'ar_balance_of_rvns_and_expns', 'ar_profit_before_tax', 'ar_taxes',
       'ar_other_profit_and_losses', 'ar_net_profit', 'ab_immobilized_assets',
       'ab_mobile_current_assets', 'ab_inventory', 'ab_accounts_receivable',
       'ab_other_current_assets', 'ab_cash_and_securities', 'ab_losses',
       'ab_own_capital', 'ab_borrowed_capital', 'ab_long_term_liabilities',
       'ab_short_term_borrowing', 'ab_accounts_payable', 'ab_other_borrowings',
       'bus_age', 'ogrn_age', 'adr_actual_age', 'head_actual_age',
       'cap_actual_age', 'ul_staff_range', 'ul_capital_sum', 'ul_founders_cnt',
       'ul_branch_cnt', 'ul_strategic_flg', 'ul_systematizing_flg',
       'default_12m', 'losses=margin'],
      dtype='object')

In [11]:
a = train_df.drop(['record_id','ogrn_age', 'adr_actual_age', 'head_actual_age',
              'ul_branch_cnt', 'ul_strategic_flg', 'ul_systematizing_flg'], axis =1)
a


,ar_revenue,ar_total_expenses,ar_sale_cost,ar_selling_expenses,ar_management_expenses,ar_sale_profit,ar_balance_of_rvns_and_expns,ar_profit_before_tax,ar_taxes,ar_other_profit_and_losses,...,ab_short_term_borrowing,ab_accounts_payable,ab_other_borrowings,bus_age,cap_actual_age,ul_staff_range,ul_capital_sum,ul_founders_cnt,default_12m,losses=margin
0,52996000.0,48344000.0,48344000.0,0.0,0.0,4652000.0,-2608000.0,2044000.0,846000.0,0.0,...,11349000.0,4523000.0,0.0,198,3,2,50000.0,2,0,1
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,256,12,2,10000.0,5,0,0
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,33,28,2,10000.0,2,0,0
3,32049000.0,29568000.0,29568000.0,0.0,0.0,2481000.0,-260000.0,2221000.0,302000.0,0.0,...,3652000.0,0.0,0.0,185,8,2,300000.0,1,0,1
4,16458000.0,14935000.0,14935000.0,0.0,0.0,1523000.0,-274000.0,1249000.0,538000.0,0.0,...,0.0,1279000.0,0.0,220,27,2,10000.0,2,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32390,28264000.0,27911000.0,27911000.0,0.0,0.0,353000.0,-68000.0,285000.0,57000.0,0.0,...,0.0,13826000.0,0.0,71,5,2,10000.0,1,0,1
32391,25017000.0,24381000.0,24381000.0,0.0,0.0,636000.0,0.0,636000.0,0.0,0.0,...,0.0,182000.0,0.0,168,25,2,600000.0,1,0,1
32392,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,11,11,2,10000.0,1,0,0
32393,136162000.0,135088000.0,135088000.0,0.0,0.0,1074000.0,-719000.0,355000.0,267000.0,0.0,...,3676000.0,301000.0,0.0,70,13,0,11000.0,1,1,1


In [12]:
b = a.drop(['ar_total_expenses', 'ar_sale_cost',
       'ar_selling_expenses', 'ar_management_expenses','ar_other_profit_and_losses', 'ar_balance_of_rvns_and_expns', 'ar_profit_before_tax'], axis = 1)
b

,ar_revenue,ar_sale_profit,ar_taxes,ar_net_profit,ab_immobilized_assets,ab_mobile_current_assets,ab_inventory,ab_accounts_receivable,ab_other_current_assets,ab_cash_and_securities,...,ab_short_term_borrowing,ab_accounts_payable,ab_other_borrowings,bus_age,cap_actual_age,ul_staff_range,ul_capital_sum,ul_founders_cnt,default_12m,losses=margin
0,52996000.0,4652000.0,846000.0,1198000.0,0.0,7240000.0,4988000.0,2252000.0,0.0,255000.0,...,11349000.0,4523000.0,0.0,198,3,2,50000.0,2,0,1
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,256,12,2,10000.0,5,0,0
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,33,28,2,10000.0,2,0,0
3,32049000.0,2481000.0,302000.0,1919000.0,0.0,4245000.0,4082000.0,163000.0,0.0,25000.0,...,3652000.0,0.0,0.0,185,8,2,300000.0,1,0,1
4,16458000.0,1523000.0,538000.0,711000.0,0.0,8950000.0,8606000.0,344000.0,0.0,492000.0,...,0.0,1279000.0,0.0,220,27,2,10000.0,2,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32390,28264000.0,353000.0,57000.0,228000.0,0.0,10147000.0,3546000.0,6601000.0,0.0,639000.0,...,0.0,13826000.0,0.0,71,5,2,10000.0,1,0,1
32391,25017000.0,636000.0,0.0,636000.0,3000.0,2977000.0,2046000.0,931000.0,0.0,7000.0,...,0.0,182000.0,0.0,168,25,2,600000.0,1,0,1
32392,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,11,11,2,10000.0,1,0,0
32393,136162000.0,1074000.0,267000.0,88000.0,0.0,20357000.0,0.0,20357000.0,0.0,32000.0,...,3676000.0,301000.0,0.0,70,13,0,11000.0,1,1,1


In [13]:
c = b.drop(['ab_long_term_liabilities','ab_short_term_borrowing', 'ab_accounts_payable',
            'ab_other_borrowings', 'ab_immobilized_assets', 'ab_other_current_assets','ab_mobile_current_assets'], axis = 1)
with_ar_df = c.dropna()
with_ar_df 

,ar_revenue,ar_sale_profit,ar_taxes,ar_net_profit,ab_inventory,ab_accounts_receivable,ab_cash_and_securities,ab_losses,ab_own_capital,ab_borrowed_capital,bus_age,cap_actual_age,ul_staff_range,ul_capital_sum,ul_founders_cnt,default_12m,losses=margin
0,52996000.0,4652000.0,846000.0,1198000.0,4988000.0,2252000.0,255000.0,1198000.0,663000.0,15872000.0,198,3,2,50000.0,2,0,1
3,32049000.0,2481000.0,302000.0,1919000.0,4082000.0,163000.0,25000.0,1919000.0,2219000.0,3652000.0,185,8,2,300000.0,1,0,1
4,16458000.0,1523000.0,538000.0,711000.0,8606000.0,344000.0,492000.0,711000.0,12439000.0,1279000.0,220,27,2,10000.0,2,0,1
9,125432000.0,2777000.0,579000.0,2135000.0,8263000.0,16417000.0,106000.0,2135000.0,11267000.0,11912000.0,170,15,0,10000.0,2,0,1
10,59570000.0,239000.0,38000.0,104000.0,23189000.0,14815000.0,449000.0,104000.0,10000.0,38443000.0,29,4,2,10000.0,2,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32388,37527000.0,5900000.0,994000.0,0.0,2828000.0,2264000.0,928000.0,0.0,20000.0,11493000.0,83,2,2,25000.0,1,0,1
32390,28264000.0,353000.0,57000.0,228000.0,3546000.0,6601000.0,639000.0,228000.0,-1597000.0,13826000.0,71,5,2,10000.0,1,0,1
32391,25017000.0,636000.0,0.0,636000.0,2046000.0,931000.0,7000.0,636000.0,2805000.0,182000.0,168,25,2,600000.0,1,0,1
32393,136162000.0,1074000.0,267000.0,88000.0,0.0,20357000.0,32000.0,88000.0,1412000.0,3977000.0,70,13,0,11000.0,1,1,1


#Выбор лучшей модели(df_without_ar)

In [14]:
X_train, X_test, y_train, y_test = train_test_split(without_ar_df, without_ar_target, test_size = 0.33, random_state = 42, shuffle = True)

In [15]:
rf = RandomForestClassifier()

scores = cross_val_score(rf, X_train, y_train.values.ravel(), cv = 5, scoring='roc_auc').mean()

In [16]:
scores

0.5817040583836924

In [17]:
hyp_param = {'n_estimators': [10, 25, 50, 100],
             'max_depth': [5, 8, 12, 24]}
cross_val = GridSearchCV(rf, hyp_param, cv = 5)
cross_val.fit(X_train, y_train.values.ravel())

GridSearchCV(cv=5, estimator=RandomForestClassifier(),
             param_grid={'max_depth': [5, 8, 12, 24],
                         'n_estimators': [10, 25, 50, 100]})

In [18]:
def results(results):
  print('Optimal hyppar: ()\n'.format(results.best_params_))
  means = results.cv_results_['mean_test_score']
  stds = results.cv_results_['std_test_score']
  for mean, std, params in zip(means, stds, results.cv_results_['params']):
    print('Mean {} Standart Deviation {} Hyperparameters {}'.format(round(mean, 2), round(std * 2, 3), params))

In [19]:
results(cross_val)

Optimal hyppar: ()

Mean 0.91 Standart Deviation 0.0 Hyperparameters {'max_depth': 5, 'n_estimators': 10}
Mean 0.91 Standart Deviation 0.0 Hyperparameters {'max_depth': 5, 'n_estimators': 25}
Mean 0.91 Standart Deviation 0.0 Hyperparameters {'max_depth': 5, 'n_estimators': 50}
Mean 0.91 Standart Deviation 0.0 Hyperparameters {'max_depth': 5, 'n_estimators': 100}
Mean 0.91 Standart Deviation 0.001 Hyperparameters {'max_depth': 8, 'n_estimators': 10}
Mean 0.91 Standart Deviation 0.0 Hyperparameters {'max_depth': 8, 'n_estimators': 25}
Mean 0.91 Standart Deviation 0.0 Hyperparameters {'max_depth': 8, 'n_estimators': 50}
Mean 0.91 Standart Deviation 0.0 Hyperparameters {'max_depth': 8, 'n_estimators': 100}
Mean 0.91 Standart Deviation 0.004 Hyperparameters {'max_depth': 12, 'n_estimators': 10}
Mean 0.91 Standart Deviation 0.002 Hyperparameters {'max_depth': 12, 'n_estimators': 25}
Mean 0.91 Standart Deviation 0.001 Hyperparameters {'max_depth': 12, 'n_estimators': 50}
Mean 0.91 Standart De

In [20]:
rf1 = RandomForestClassifier(max_depth = 8, n_estimators = 50)
rf1.fit(X_test, y_test)

rf2 = RandomForestClassifier(max_depth = 15, n_estimators = 100)
rf2.fit(X_test, y_test)

rf3 = RandomForestClassifier(max_depth = 24, n_estimators = 100)
rf3.fit(X_test, y_test)


RandomForestClassifier(max_depth=24)

In [21]:
for mdl in [rf1, rf2, rf3]:
  y_pred = mdl.predict(X_test)
  accuracy = round(accuracy_score(y_test, y_pred), 3)
  precision = round(precision_score(y_test, y_pred), 3)
  recall = round(recall_score(y_test, y_pred), 3)

  print('Max depth: {}|| Estimators: {} || Accuracy: {} || Precision: {} || Recall: {}'.format(mdl.max_depth,
                                                                                               mdl.n_estimators,
                                                                                               accuracy,
                                                                                               precision,
                                                                                               recall))

Max depth: 8|| Estimators: 50 || Accuracy: 0.92 || Precision: 1.0 || Recall: 0.02
Max depth: 15|| Estimators: 100 || Accuracy: 0.951 || Precision: 0.982 || Recall: 0.4
Max depth: 24|| Estimators: 100 || Accuracy: 0.972 || Precision: 0.965 || Recall: 0.683


In [22]:
cv = cross_val_score(rf3, X_train, y_train, cv=5, scoring='roc_auc').mean()
cv

0.5778097884106786

In [23]:

logit = LogisticRegression(C=1,solver='lbfgs',max_iter=1000)
cv_baseline = cross_val_score(logit, X_train, y_train, cv=5, scoring='roc_auc').mean()
cv_baseline

0.6317919047456441

In [24]:
import xgboost as xgb
xgb_cl1 = xgb.XGBClassifier(objective="binary:logistic",
    max_depth= 5,
    learning_rate = 0.001,
    gamma= 0.2,
    reg_lambda=  1,
    scale_pos_weight= 6,
    max_features = 3,
    subsample= 0.8,
    colsample_bytree= 0.5)
# xgb_cl.fit(X_train, y_train)
# preds = xgb_cl.predict(X_test)
cv = cross_val_score(xgb_cl1, X_train, y_train, cv=5, scoring='roc_auc').mean()
cv

0.6351278501555774

#Выбор лучшей модели(with_ar_df)

In [25]:
X_train, X_test, y_train, y_test = train_test_split(with_ar_df.drop('default_12m', axis = 1), with_ar_df['default_12m'], 
                                                    test_size = 0.33, 
                                                    random_state = 42,
                                                    shuffle = True)

In [26]:
rf = RandomForestClassifier()

scores = cross_val_score(rf, X_train, y_train.values.ravel(), cv = 5, scoring='roc_auc').mean()

In [27]:
scores

0.7167431694424126

In [28]:
def results(results):
  print('Optimal hyppar: ()\n'.format(results.best_params_))
  means = results.cv_results_['mean_test_score']
  stds = results.cv_results_['std_test_score']
  for mean, std, params in zip(means, stds, results.cv_results_['params']):
    print('Mean {} Standart Deviation {} Hyperparameters {}'.format(round(mean, 2), round(std * 2, 3), params))

In [29]:
results(cross_val)

Optimal hyppar: ()

Mean 0.91 Standart Deviation 0.0 Hyperparameters {'max_depth': 5, 'n_estimators': 10}
Mean 0.91 Standart Deviation 0.0 Hyperparameters {'max_depth': 5, 'n_estimators': 25}
Mean 0.91 Standart Deviation 0.0 Hyperparameters {'max_depth': 5, 'n_estimators': 50}
Mean 0.91 Standart Deviation 0.0 Hyperparameters {'max_depth': 5, 'n_estimators': 100}
Mean 0.91 Standart Deviation 0.001 Hyperparameters {'max_depth': 8, 'n_estimators': 10}
Mean 0.91 Standart Deviation 0.0 Hyperparameters {'max_depth': 8, 'n_estimators': 25}
Mean 0.91 Standart Deviation 0.0 Hyperparameters {'max_depth': 8, 'n_estimators': 50}
Mean 0.91 Standart Deviation 0.0 Hyperparameters {'max_depth': 8, 'n_estimators': 100}
Mean 0.91 Standart Deviation 0.004 Hyperparameters {'max_depth': 12, 'n_estimators': 10}
Mean 0.91 Standart Deviation 0.002 Hyperparameters {'max_depth': 12, 'n_estimators': 25}
Mean 0.91 Standart Deviation 0.001 Hyperparameters {'max_depth': 12, 'n_estimators': 50}
Mean 0.91 Standart De

In [30]:
rf1 = RandomForestClassifier(max_depth = 24, n_estimators = 25)
rf1.fit(X_test, y_test)

rf2 = RandomForestClassifier(max_depth = 24, n_estimators = 50)
rf2.fit(X_test, y_test)

rf3 = RandomForestClassifier(max_depth = 24, n_estimators = 100)
rf3.fit(X_test, y_test)

RandomForestClassifier(max_depth=24)

In [31]:
for mdl in [rf1, rf2, rf3]:
  y_pred = mdl.predict(X_test)
  accuracy = round(accuracy_score(y_test, y_pred), 3)
  precision = round(precision_score(y_test, y_pred), 3)
  recall = round(recall_score(y_test, y_pred), 3)

  print('Max depth: {}|| Estimators: {} || Accuracy: {} || Precision: {} || Recall: {}'.format(mdl.max_depth,
                                                                                               mdl.n_estimators,
                                                                                               accuracy,
                                                                                               precision,
                                                                                               recall))

Max depth: 24|| Estimators: 25 || Accuracy: 0.997 || Precision: 1.0 || Recall: 0.931
Max depth: 24|| Estimators: 50 || Accuracy: 0.999 || Precision: 0.996 || Recall: 0.992
Max depth: 24|| Estimators: 100 || Accuracy: 1.0 || Precision: 0.996 || Recall: 0.996


In [32]:
cv = cross_val_score(rf3, X_train, y_train, cv=5, scoring='roc_auc').mean()
cv

0.7275341885090393

In [33]:
xgb_cl2 = xgb.XGBClassifier(
    max_depth= 12,
    learning_rate = 0.001,
    gamma= 0.2,
    reg_lambda=  1,
    scale_pos_weight= 8,
    subsample= 0.8,
    colsample_bytree= 0.5)
# xgb_cl.fit(X_train, y_train)
# preds = xgb_cl.predict(X_test)
cv = cross_val_score(xgb_cl2, X_train, y_train, cv=5, scoring='roc_auc').mean()
cv

0.7395935688476135

#подготовка тестовых данных


In [34]:
with_ar_df.columns

Index(['ar_revenue', 'ar_sale_profit', 'ar_taxes', 'ar_net_profit',
       'ab_inventory', 'ab_accounts_receivable', 'ab_cash_and_securities',
       'ab_losses', 'ab_own_capital', 'ab_borrowed_capital', 'bus_age',
       'cap_actual_age', 'ul_staff_range', 'ul_capital_sum', 'ul_founders_cnt',
       'default_12m', 'losses=margin'],
      dtype='object')

In [35]:
test_with_df = test_df.dropna()
test_with_df = test_with_df[['record_id', 'ar_revenue', 'ar_sale_profit', 'ar_taxes', 'ar_net_profit',
       'ab_inventory', 'ab_accounts_receivable', 'ab_cash_and_securities',
       'ab_losses', 'ab_own_capital', 'ab_borrowed_capital', 'bus_age',
       'cap_actual_age', 'ul_staff_range', 'ul_capital_sum', 'ul_founders_cnt','losses=margin']]

a = test_df[test_df.isnull().any(1)]
test_without_df= a.loc[:, features] 

In [36]:

test_with_df

,record_id,ar_revenue,ar_sale_profit,ar_taxes,ar_net_profit,ab_inventory,ab_accounts_receivable,ab_cash_and_securities,ab_losses,ab_own_capital,ab_borrowed_capital,bus_age,cap_actual_age,ul_staff_range,ul_capital_sum,ul_founders_cnt,losses=margin
0,196,39103000.0,190000.0,284000.0,47000.0,2451000.0,3558000.0,677000.0,47000.0,1178000.0,6130000.0,156,3,2,100000.0,1,1
2,2813,54174000.0,3245000.0,500000.0,1467000.0,8037000.0,138000.0,237000.0,1467000.0,6835000.0,3906000.0,244,20,2,250000.0,2,1
3,4385,1904000.0,225000.0,0.0,137000.0,4314000.0,1476000.0,35000.0,137000.0,5691000.0,540000.0,272,9,2,10000.0,2,1
4,6479,225584000.0,14899000.0,445000.0,1739000.0,46006000.0,19479000.0,956000.0,1739000.0,12408000.0,47017000.0,254,3,0,1700598.0,84,1
5,6726,14447000.0,604000.0,0.0,1004000.0,1451000.0,0.0,0.0,1004000.0,1562000.0,447000.0,92,14,1,558967.0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
189,260260,25122000.0,13619000.0,271000.0,14791000.0,29910000.0,4977000.0,258000.0,14791000.0,31065000.0,5498000.0,95,24,2,2510000.0,1,1
192,263728,44096000.0,401000.0,105000.0,275000.0,10332000.0,7915000.0,2995000.0,275000.0,1524000.0,17709000.0,60,23,2,10000.0,2,1
193,264010,8373000.0,75000.0,21000.0,43000.0,1121000.0,999000.0,160000.0,43000.0,74000.0,2206000.0,37,5,2,10000.0,1,1
195,265577,21287000.0,2449000.0,483000.0,1843000.0,6444000.0,1979000.0,2534000.0,1846000.0,4424000.0,6714000.0,47,32,2,10000.0,1,0


In [37]:

test_without_df

,bus_age,ul_staff_range,ul_founders_cnt,ul_capital_sum,head_actual_age
1,216,2,1,100000.0,5
6,98,0,1,10000.0,28
7,43,2,1,10000.0,28
8,16,2,1,10000.0,11
9,33,2,1,10000.0,2
...,...,...,...,...,...
191,101,2,1,10000.0,7
194,241,2,1,0.0,6
196,283,2,1,0.0,6
197,22,2,1,20000.0,7


#Сведение двух моделей

In [38]:
cutoff_with = with_ar_df['default_12m'].mean()
cutoff_with

0.04614492753623188

In [39]:
cutoff_without = without_ar_target.mean()
cutoff_without

0.08550676791020138

In [40]:
logit.fit(without_ar_df, without_ar_target)
rf3.fit(with_ar_df.drop('default_12m', axis = 1), with_ar_df['default_12m'])

RandomForestClassifier(max_depth=24)

In [41]:
answ_df1 = a[['record_id']]
answ_df2 = test_with_df[['record_id']]


In [42]:
answ_df1['id'] = answ_df1['record_id']
answ_df2['id'] = answ_df2['record_id']

<ipython-input-42-4d46e85f0361>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  answ_df1['id'] = answ_df1['record_id']
<ipython-input-42-4d46e85f0361>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  answ_df2['id'] = answ_df2['record_id']


In [43]:
answ_df1.drop('record_id',axis=1, inplace=True)
answ_df2.drop('record_id',axis=1, inplace=True)

/usr/local/lib/python3.8/dist-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [44]:
answ_df1['predict'] = (logit.predict_proba(test_without_df)[:,1]>cutoff_with).astype(int)
answ_df2['predict'] = (rf3.predict_proba(test_with_df.drop('record_id', axis = 1))[:,1]>cutoff_without).astype(int)

<ipython-input-44-5d0d31cc9be8>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  answ_df1['predict'] = (logit.predict_proba(test_without_df)[:,1]>cutoff_with).astype(int)
<ipython-input-44-5d0d31cc9be8>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  answ_df2['predict'] = (rf3.predict_proba(test_with_df.drop('record_id', axis = 1))[:,1]>cutoff_without).astype(int)


In [45]:
answ_df1.head()

,id,predict
1,1196,0
6,9178,0
7,13143,0
8,13547,1
9,18483,1


In [46]:
answ_df = pd.concat([answ_df1, answ_df2])
answ_df

,id,predict
1,1196,0
6,9178,0
7,13143,0
8,13547,1
9,18483,1
...,...,...
189,260260,1
192,263728,1
193,264010,0
195,265577,1


In [47]:
answ_df = answ_df.sort_index(ascending=True)


In [48]:
answ_df

,id,predict
0,196,0
1,1196,0
2,2813,0
3,4385,1
4,6479,0
...,...,...
195,265577,1
196,266648,0
197,266849,1
198,267566,0


In [49]:
answ_df['predict'].values.mean()

0.465

In [50]:
answ_df.to_csv('PD-submit.csv',index=False, sep=';')